# Задача 2: Нейронные сети и анализ тональности

Примеры использования Keras для понимания задачи и проверки результатов.

## Импорт необходимых библиотек

## Пример: Анализ тональности текстов

Основная задача - анализ тональности текстовых отзывов. Сначала подготовим данные и векторизуем тексты.

In [ ]:
# Пример данных для анализа тональности (отзывы)
sentiment_data = {
    'text': [
        # Положительные отзывы
        'Отличный фильм, очень понравился!',
        'Прекрасная книга, рекомендую всем прочитать.',
        'Качественный товар, доволен покупкой.',
        'Замечательный сервис, быстро и профессионально.',
        'Потрясающий ресторан, вкусная еда и уютная атмосфера.',
        'Великолепный спектакль, получил массу удовольствия.',
        'Отличное приложение, удобный интерфейс.',
        'Прекрасный отель, комфортные номера и хороший завтрак.',
        'Замечательный преподаватель, интересные лекции.',
        'Потрясающая музыка, слушаю постоянно.',
        'Отличный телефон, все работает быстро.',
        'Прекрасный подарок, очень доволен.',
        'Качественный кофе, ароматный и вкусный.',
        'Замечательный парк, красивая природа.',
        'Потрясающий концерт, незабываемые эмоции.',
        
        # Отрицательные отзывы
        'Ужасный фильм, полная потеря времени.',
        'Плохая книга, скучно и неинтересно.',
        'Низкое качество товара, очень разочарован.',
        'Плохой сервис, долго ждал и ничего не получил.',
        'Отвратительный ресторан, невкусная еда.',
        'Скучный спектакль, не рекомендую.',
        'Плохое приложение, постоянно глючит.',
        'Ужасный отель, грязные номера и плохое обслуживание.',
        'Скучный преподаватель, неинтересные лекции.',
        'Плохая музыка, не понравилась.',
        'Плохой телефон, постоянно зависает.',
        'Разочаровался в подарке, не то что ожидал.',
        'Плохой кофе, безвкусный и холодный.',
        'Запущенный парк, грязно и неухоженно.',
        'Скучный концерт, не оправдал ожиданий.',
        
        # Нейтральные отзывы (для мультиклассовой классификации)
        'Обычный фильм, ничего особенного.',
        'Стандартная книга, читается нормально.',
        'Обычный товар, соответствует описанию.',
        'Нормальный сервис, без особых претензий.',
        'Обычный ресторан, стандартное меню.',
        'Средний спектакль, можно посмотреть.',
        'Обычное приложение, работает как надо.',
        'Стандартный отель, без изысков.',
        'Обычный преподаватель, стандартные лекции.',
        'Средняя музыка, ничего особенного.',
    ],
    'sentiment': [
        # Положительные (1)
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        # Отрицательные (0)
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        # Нейтральные (2)
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
    ]
}

df_sentiment = pd.DataFrame(sentiment_data)
print(f\"Всего отзывов: {len(df_sentiment)}\")
print(f\"Распределение тональности:\")
print(df_sentiment['sentiment'].value_counts().sort_index())
print(\"\\nПримеры отзывов:\")
print(df_sentiment.head(10))

## Векторизация текстов для анализа тональности

In [ ]:
# Векторизация текстов с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=100, stop_words=None, ngram_range=(1, 2))
X_sentiment = vectorizer.fit_transform(df_sentiment['text']).toarray()
y_sentiment = df_sentiment['sentiment'].values

print(f\"Размерность векторизованных данных: {X_sentiment.shape}\")
print(f\"Количество признаков (слов и биграмм): {X_sentiment.shape[1]}\")

# Разделение на train/test
X_train_sent, X_test_sent, y_train_sent, y_test_sent = train_test_split(
    X_sentiment, y_sentiment, test_size=0.3, random_state=42, stratify=y_sentiment
)

# Нормализация (опционально, но часто помогает)
scaler_sent = StandardScaler()
X_train_sent_scaled = scaler_sent.fit_transform(X_train_sent)
X_test_sent_scaled = scaler_sent.transform(X_test_sent)

print(f\"\\nОбучающая выборка: {X_train_sent_scaled.shape}\")
print(f\"Тестовая выборка: {X_test_sent_scaled.shape}\")
print(f\"Классы в обучающей выборке: {np.unique(y_train_sent, return_counts=True)}\")

## Пример: Бинарная классификация тональности (положительная/отрицательная)

In [ ]:
# Фильтруем только положительные и отрицательные отзывы (бинарная классификация)
mask_binary = (y_train_sent != 2) & (y_test_sent != 2)
X_train_bin_sent = X_train_sent_scaled[y_train_sent != 2]
y_train_bin_sent = y_train_sent[y_train_sent != 2]
X_test_bin_sent = X_test_sent_scaled[y_test_sent != 2]
y_test_bin_sent = y_test_sent[y_test_sent != 2]

print(f\"Бинарная классификация: {len(X_train_bin_sent)} обучающих, {len(X_test_bin_sent)} тестовых\")
print(f\"Классы: {np.unique(y_train_bin_sent)}\")

# Модель для бинарной классификации тональности
model_sentiment_binary = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_bin_sent.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Бинарная классификация
])

model_sentiment_binary.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Logloss для бинарной классификации
    metrics=['accuracy']
)

# Early stopping
early_stopping_sent = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Обучение
history_sent_binary = model_sentiment_binary.fit(
    X_train_bin_sent, y_train_bin_sent,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping_sent],
    verbose=1
)

# Предсказания
y_pred_sent_binary = (model_sentiment_binary.predict(X_test_bin_sent) > 0.5).astype(int).flatten()
y_pred_proba_sent_binary = model_sentiment_binary.predict(X_test_bin_sent).flatten()

# Метрики
print(\"\\nМетрики для анализа тональности (бинарная классификация):\")
print(f\"F1-score: {f1_score(y_test_bin_sent, y_pred_sent_binary):.4f}\")
print(f\"Precision: {precision_score(y_test_bin_sent, y_pred_sent_binary):.4f}\")
print(f\"Recall: {recall_score(y_test_bin_sent, y_pred_sent_binary):.4f}\")

# Примеры предсказаний
print(\"\\nПримеры предсказаний:\")
test_texts = df_sentiment.loc[y_test_sent != 2, 'text'].values[:5]
for i, text in enumerate(test_texts[:5]):
    print(f\"\\nТекст: {text}\")
    print(f\"Истинная тональность: {'Положительная' if y_test_bin_sent[i] == 1 else 'Отрицательная'}\")
    print(f\"Предсказанная тональность: {'Положительная' if y_pred_sent_binary[i] == 1 else 'Отрицательная'}\")
    print(f\"Вероятность положительной: {y_pred_proba_sent_binary[i]:.4f}\")

## Пример: Мультиклассовая классификация тональности (положительная/отрицательная/нейтральная)

## Пример: Регрессия - предсказание рейтинга отзывов (0-10)

In [ ]:
# Данные для регрессии рейтингов
# Создаем рейтинги от 0 до 10 на основе тех же текстов
rating_data = {
    'text': df_sentiment['text'].tolist(),
    'rating': [
        # Положительные отзывы - высокие рейтинги (7-10)
        9, 8, 9, 8, 10, 9, 8, 9, 8, 9, 8, 9, 8, 9, 10,
        # Отрицательные отзывы - низкие рейтинги (0-3)
        1, 2, 1, 2, 0, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0,
        # Нейтральные отзывы - средние рейтинги (4-6)
        5, 6, 5, 6, 5, 6, 5, 6, 5, 6,
    ]
}

df_rating = pd.DataFrame(rating_data)
print(f\"Всего отзывов с рейтингами: {len(df_rating)}\")
print(f\"Диапазон рейтингов: [{df_rating['rating'].min()}, {df_rating['rating'].max()}]\")
print(f\"Средний рейтинг: {df_rating['rating'].mean():.2f}\")
print(f\"Распределение рейтингов:\")
print(df_rating['rating'].value_counts().sort_index())

# Векторизация текстов (используем тот же vectorizer или создаем новый)
vectorizer_rating = TfidfVectorizer(max_features=100, stop_words=None, ngram_range=(1, 2))
X_rating = vectorizer_rating.fit_transform(df_rating['text']).toarray()
y_rating = df_rating['rating'].values.astype(float)

# Разделение на train/test
X_train_rating, X_test_rating, y_train_rating, y_test_rating = train_test_split(
    X_rating, y_rating, test_size=0.3, random_state=42
)

# Нормализация признаков
scaler_rating = StandardScaler()
X_train_rating_scaled = scaler_rating.fit_transform(X_train_rating)
X_test_rating_scaled = scaler_rating.transform(X_test_rating)

# Нормализация рейтингов (от 0 до 10 -> от 0 до 1 для лучшей сходимости)
y_train_rating_scaled = y_train_rating / 10.0
y_test_rating_scaled = y_test_rating / 10.0

print(f\"\\nОбучающая выборка: {X_train_rating_scaled.shape}\")
print(f\"Тестовая выборка: {X_test_rating_scaled.shape}\")
print(f\"Диапазон рейтингов в обучающей выборке: [{y_train_rating.min():.1f}, {y_train_rating.max():.1f}]\")

In [ ]:
# Модель для регрессии рейтингов
model_rating = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_rating_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Sigmoid для ограничения выхода от 0 до 1
])

# Компиляция
model_rating.compile(
    optimizer='adam',
    loss='mse',  # Mean Squared Error для регрессии
    metrics=['mae']
)

# Early stopping
early_stopping_rating = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Обучение
history_rating = model_rating.fit(
    X_train_rating_scaled, y_train_rating_scaled,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping_rating],
    verbose=1
)

# Предсказания (обратная нормализация: умножаем на 10 и ограничиваем от 0 до 10)
y_pred_rating_scaled = model_rating.predict(X_test_rating_scaled).flatten()
y_pred_rating = np.clip(y_pred_rating_scaled * 10.0, 0, 10)

# Метрики для регрессии рейтингов
mae_rating = mean_absolute_error(y_test_rating, y_pred_rating)
rmse_rating = np.sqrt(mean_squared_error(y_test_rating, y_pred_rating))
mape_rating = np.mean(np.abs((y_test_rating - y_pred_rating) / (y_test_rating + 1e-8))) * 100  # +1e-8 чтобы избежать деления на 0

print(\"\\nМетрики для регрессии рейтингов:\")
print(f\"MAE: {mae_rating:.4f} (средняя ошибка в баллах)\")
print(f\"RMSE: {rmse_rating:.4f} (корень из средней квадратичной ошибки)\")
print(f\"MAPE: {mape_rating:.4f}% (средняя абсолютная процентная ошибка)\")

# Примеры предсказаний
print(\"\\nПримеры предсказаний рейтингов:\")
test_texts_rating = df_rating.loc[y_test_rating.index if hasattr(y_test_rating, 'index') else range(len(y_test_rating)), 'text'].values[:5]
for i, text in enumerate(test_texts_rating[:5]):
    print(f\"\\nТекст: {text}\")
    print(f\"Истинный рейтинг: {y_test_rating[i]:.1f}/10\")
    print(f\"Предсказанный рейтинг: {y_pred_rating[i]:.1f}/10\")
    print(f\"Ошибка: {abs(y_test_rating[i] - y_pred_rating[i]):.2f} баллов\")

In [ ]:
# Мультиклассовая классификация (3 класса: положительная, отрицательная, нейтральная)
# One-hot encoding для мультиклассовой классификации
y_train_sent_onehot = keras.utils.to_categorical(y_train_sent, num_classes=3)
y_test_sent_onehot = keras.utils.to_categorical(y_test_sent, num_classes=3)

print(f\"Классы: {np.unique(y_train_sent)}\")
print(f\"Форма y_train_onehot: {y_train_sent_onehot.shape}\")

# Модель для мультиклассовой классификации тональности
model_sentiment_multi = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_sent_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(3, activation='softmax')  # Softmax для мультиклассовой классификации
])

model_sentiment_multi.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Categorical cross-entropy для мультиклассовой
    metrics=['accuracy']
)

# Early stopping
early_stopping_sent_multi = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Обучение
history_sent_multi = model_sentiment_multi.fit(
    X_train_sent_scaled, y_train_sent_onehot,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping_sent_multi],
    verbose=1
)

# Предсказания
y_pred_proba_sent_multi = model_sentiment_multi.predict(X_test_sent_scaled)
y_pred_sent_multi = np.argmax(y_pred_proba_sent_multi, axis=1)

# Метрики (для мультиклассовой используем average='weighted' или 'macro')
print(\"\\nМетрики для анализа тональности (мультиклассовая классификация):\")
print(f\"F1-score (weighted): {f1_score(y_test_sent, y_pred_sent_multi, average='weighted'):.4f}\")
print(f\"Precision (weighted): {precision_score(y_test_sent, y_pred_sent_multi, average='weighted'):.4f}\")
print(f\"Recall (weighted): {recall_score(y_test_sent, y_pred_sent_multi, average='weighted'):.4f}\")

# Примеры предсказаний
sentiment_labels = {0: 'Отрицательная', 1: 'Положительная', 2: 'Нейтральная'}
print(\"\\nПримеры предсказаний:\")
test_texts_multi = df_sentiment.loc[y_test_sent.index if hasattr(y_test_sent, 'index') else range(len(y_test_sent)), 'text'].values[:5]
for i, text in enumerate(test_texts_multi[:5]):
    print(f\"\\nТекст: {text}\")
    print(f\"Истинная тональность: {sentiment_labels[y_test_sent[i]]}\")
    print(f\"Предсказанная тональность: {sentiment_labels[y_pred_sent_multi[i]]}\")
    print(f\"Вероятности: {dict(zip(['Отрицательная', 'Положительная', 'Нейтральная'], y_pred_proba_sent_multi[i]))}\")

## Генерация данных для классификации (общие примеры)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    f1_score, precision_score, recall_score,
    mean_absolute_error, mean_squared_error
)
from sklearn.preprocessing import StandardScaler

# Keras для примеров
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

## Генерация данных для классификации

In [ ]:
# Генерация сбалансированной выборки для бинарной классификации
X_binary, y_binary = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    n_clusters_per_class=1,
    random_state=42,
    class_sep=1.0
)

# Разделение на train/test
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    X_binary, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# Нормализация данных
scaler_bin = StandardScaler()
X_train_bin_scaled = scaler_bin.fit_transform(X_train_bin)
X_test_bin_scaled = scaler_bin.transform(X_test_bin)

print(f"Размер обучающей выборки: {X_train_bin_scaled.shape}")
print(f"Размер тестовой выборки: {X_test_bin_scaled.shape}")
print(f"Классы: {np.unique(y_train_bin)}")

## Пример 1: Бинарная классификация с Keras (базовая модель)

In [ ]:
# Создание модели
model_binary = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_bin_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Бинарная классификация
])

# Компиляция модели
model_binary.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Logloss для бинарной классификации
    metrics=['accuracy']
)

# Обучение
history_binary = model_binary.fit(
    X_train_bin_scaled, y_train_bin,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Предсказания
y_pred_binary = (model_binary.predict(X_test_bin_scaled) > 0.5).astype(int)
y_pred_proba_binary = model_binary.predict(X_test_bin_scaled)

# Метрики
print("\nМетрики для бинарной классификации:")
print(f"F1-score: {f1_score(y_test_bin, y_pred_binary):.4f}")
print(f"Precision: {precision_score(y_test_bin, y_pred_binary):.4f}")
print(f"Recall: {recall_score(y_test_bin, y_pred_binary):.4f}")

## Пример 2: Бинарная классификация с Dropout и Early Stopping

In [ ]:
# Создание модели с dropout
model_binary_dropout = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_bin_scaled.shape[1],)),
    layers.Dropout(0.3),  # Dropout с вероятностью 0.3
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),  # Dropout с вероятностью 0.2
    layers.Dense(1, activation='sigmoid')
])

# Компиляция
model_binary_dropout.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Обучение с early stopping
history_dropout = model_binary_dropout.fit(
    X_train_bin_scaled, y_train_bin,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Предсказания
y_pred_dropout = (model_binary_dropout.predict(X_test_bin_scaled) > 0.5).astype(int)

# Метрики
print("\nМетрики с dropout и early stopping:")
print(f"F1-score: {f1_score(y_test_bin, y_pred_dropout):.4f}")
print(f"Precision: {precision_score(y_test_bin, y_pred_dropout):.4f}")
print(f"Recall: {recall_score(y_test_bin, y_pred_dropout):.4f}")

## Пример 3: Мультиклассовая классификация

In [ ]:
# Генерация данных для мультиклассовой классификации
X_multi, y_multi = make_classification(
    n_samples=1500,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=3,  # 3 класса
    n_clusters_per_class=1,
    random_state=42
)

# Разделение на train/test
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

# Нормализация
scaler_multi = StandardScaler()
X_train_multi_scaled = scaler_multi.fit_transform(X_train_multi)
X_test_multi_scaled = scaler_multi.transform(X_test_multi)

# One-hot encoding для мультиклассовой классификации
y_train_multi_onehot = keras.utils.to_categorical(y_train_multi, num_classes=3)
y_test_multi_onehot = keras.utils.to_categorical(y_test_multi, num_classes=3)

print(f"Классы: {np.unique(y_train_multi)}")
print(f"Форма y_train_onehot: {y_train_multi_onehot.shape}")

In [ ]:
# Модель для мультиклассовой классификации
model_multi = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_multi_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(3, activation='softmax')  # Softmax для мультиклассовой классификации
])

# Компиляция
model_multi.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Categorical cross-entropy для мультиклассовой
    metrics=['accuracy']
)

# Early stopping
early_stopping_multi = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Обучение
history_multi = model_multi.fit(
    X_train_multi_scaled, y_train_multi_onehot,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping_multi],
    verbose=1
)

# Предсказания
y_pred_proba_multi = model_multi.predict(X_test_multi_scaled)
y_pred_multi = np.argmax(y_pred_proba_multi, axis=1)

# Метрики (для мультиклассовой используем average='weighted' или 'macro')
print("\nМетрики для мультиклассовой классификации:")
print(f"F1-score (weighted): {f1_score(y_test_multi, y_pred_multi, average='weighted'):.4f}")
print(f"Precision (weighted): {precision_score(y_test_multi, y_pred_multi, average='weighted'):.4f}")
print(f"Recall (weighted): {recall_score(y_test_multi, y_pred_multi, average='weighted'):.4f}")

## Пример 4: Регрессия

In [ ]:
# Генерация данных для регрессии
X_reg, y_reg = make_regression(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    noise=10.0,
    random_state=42
)

# Разделение на train/test
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Нормализация
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# Нормализация целевой переменной
y_scaler = StandardScaler()
y_train_reg_scaled = y_scaler.fit_transform(y_train_reg.reshape(-1, 1)).flatten()
y_test_reg_scaled = y_scaler.transform(y_test_reg.reshape(-1, 1)).flatten()

print(f"Размер обучающей выборки: {X_train_reg_scaled.shape}")
print(f"Диапазон y_train: [{y_train_reg.min():.2f}, {y_train_reg.max():.2f}]")

In [ ]:
# Модель для регрессии
model_reg = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_reg_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)  # Один выход для регрессии (без активации)
])

# Компиляция
model_reg.compile(
    optimizer='adam',
    loss='mse',  # Mean Squared Error для регрессии
    metrics=['mae']
)

# Early stopping
early_stopping_reg = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Обучение
history_reg = model_reg.fit(
    X_train_reg_scaled, y_train_reg_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping_reg],
    verbose=1
)

# Предсказания (обратная нормализация)
y_pred_reg_scaled = model_reg.predict(X_test_reg_scaled).flatten()
y_pred_reg = y_scaler.inverse_transform(y_pred_reg_scaled.reshape(-1, 1)).flatten()

# Метрики для регрессии
mae = mean_absolute_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
mape = np.mean(np.abs((y_test_reg - y_pred_reg) / y_test_reg)) * 100

print("\nМетрики для регрессии:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

## Пример 5: Визуализация процесса обучения

In [ ]:
# График loss по эпохам
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_binary.history['loss'], label='Train Loss')
plt.plot(history_binary.history['val_loss'], label='Validation Loss')
plt.title('Binary Classification - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_binary.history['accuracy'], label='Train Accuracy')
plt.plot(history_binary.history['val_accuracy'], label='Validation Accuracy')
plt.title('Binary Classification - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Пример 6: Cross-validation

In [ ]:
from sklearn.model_selection import KFold

# K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_idx, val_idx in kfold.split(X_train_bin_scaled):
    X_train_cv, X_val_cv = X_train_bin_scaled[train_idx], X_train_bin_scaled[val_idx]
    y_train_cv, y_val_cv = y_train_bin[train_idx], y_train_bin[val_idx]
    
    # Создание и обучение модели
    model_cv = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train_bin_scaled.shape[1],)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model_cv.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    model_cv.fit(
        X_train_cv, y_train_cv,
        epochs=50,
        batch_size=32,
        verbose=0
    )
    
    # Оценка на валидационной выборке
    y_pred_cv = (model_cv.predict(X_val_cv) > 0.5).astype(int)
    score = f1_score(y_val_cv, y_pred_cv)
    cv_scores.append(score)

print(f"\nCross-Validation F1-scores: {cv_scores}")
print(f"Mean CV F1-score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

## Скелет для реализации вашей нейронной сети

In [ ]:
class NeuralNetwork:
    """
    Простая полносвязная нейронная сеть.
    
    Параметры:
    ----------
    layers : list
        Список размеров слоев, например [64, 32, 1] для сети с двумя скрытыми слоями
    activation : str
        Функция активации ('relu', 'sigmoid', 'softmax')
    dropout_rate : float
        Вероятность dropout (0.0 - без dropout)
    task_type : str
        Тип задачи ('classification' или 'regression')
    num_classes : int
        Количество классов (для классификации)
    """
    
    def __init__(self, layers, activation='relu', dropout_rate=0.0, 
                 task_type='classification', num_classes=2):
        # TODO: инициализировать веса, сохранить параметры
        # Подсказка: используйте np.random.randn() для инициализации весов
        # Подсказка: для весов используйте инициализацию Xavier или He
        pass
    
    def _forward_pass(self, X):
        """
        Прямой проход (forward pass) через сеть.
        
        Параметры:
        ----------
        X : numpy array
            Входные данные
        
        Возвращает:
        --------
        activations : list
            Список активаций для каждого слоя
        """
        # TODO: реализовать forward pass
        # Подсказка: для каждого слоя вычислить z = X @ W + b, затем активацию
        # Подсказка: для dropout - умножить на маску во время обучения
        pass
    
    def _backward_pass(self, X, y, activations):
        """
        Обратный проход (backpropagation) для вычисления градиентов.
        
        Параметры:
        ----------
        X : numpy array
            Входные данные
        y : numpy array
            Целевые значения
        activations : list
            Активации из forward pass
        
        Возвращает:
        --------
        gradients : dict
            Словарь градиентов для весов и смещений
        """
        # TODO: реализовать backpropagation
        # Подсказка: начать с вычисления градиента функции потерь
        # Подсказка: распространить градиент назад через слои
        # Подсказка: использовать цепное правило (chain rule)
        pass
    
    def _compute_loss(self, y_true, y_pred):
        """
        Вычисляет функцию потерь.
        
        Параметры:
        ----------
        y_true : numpy array
            Истинные значения
        y_pred : numpy array
            Предсказанные значения
        
        Возвращает:
        --------
        loss : float
            Значение функции потерь
        """
        # TODO: реализовать logloss для классификации
        # Для бинарной классификации: -mean(y*log(y_pred) + (1-y)*log(1-y_pred))
        # Для мультиклассовой: categorical cross-entropy
        # Для регрессии: MSE или MAE
        pass
    
    def fit(self, X, y, epochs=100, batch_size=32, learning_rate=0.01,
            validation_data=None, early_stopping=False, patience=10):
        """
        Обучает нейронную сеть.
        
        Параметры:
        ----------
        X : numpy array
            Обучающие данные
        y : numpy array
            Целевые значения
        epochs : int
            Количество эпох
        batch_size : int
            Размер батча
        learning_rate : float
            Скорость обучения
        validation_data : tuple
            (X_val, y_val) для валидации
        early_stopping : bool
            Использовать ли early stopping
        patience : int
            Терпение для early stopping
        """
        # TODO: реализовать обучение
        # Подсказка: для каждой эпохи:
        #   1. Разбить данные на батчи
        #   2. Для каждого батча: forward pass -> backward pass -> обновление весов
        #   3. Вычислить loss на валидации (если есть)
        #   4. Проверить early stopping (если включен)
        pass
    
    def predict(self, X):
        """
        Предсказывает классы/значения.
        
        Параметры:
        ----------
        X : numpy array
            Входные данные
        
        Возвращает:
        --------
        predictions : numpy array
            Предсказанные классы или значения
        """
        # TODO: реализовать предсказание
        # Подсказка: выполнить forward pass (без dropout!)
        # Подсказка: для классификации - argmax или порог 0.5
        pass
    
    def predict_proba(self, X):
        """
        Предсказывает вероятности (только для классификации).
        
        Параметры:
        ----------
        X : numpy array
            Входные данные
        
        Возвращает:
        --------
        probabilities : numpy array
            Вероятности классов
        """
        # TODO: реализовать предсказание вероятностей
        # Подсказка: выполнить forward pass и вернуть выход последнего слоя
        # (для бинарной - sigmoid, для мультиклассовой - softmax)
        pass

## Пример использования вашей реализации

In [ ]:
# Пример использования (после реализации)
# nn = NeuralNetwork(
#     layers=[64, 32, 1],
#     activation='relu',
#     dropout_rate=0.3,
#     task_type='classification',
#     num_classes=2
# )
# 
# nn.fit(
#     X_train_bin_scaled, y_train_bin,
#     epochs=100,
#     batch_size=32,
#     learning_rate=0.01,
#     validation_data=(X_test_bin_scaled, y_test_bin),
#     early_stopping=True,
#     patience=10
# )
# 
# y_pred = nn.predict(X_test_bin_scaled)
# y_proba = nn.predict_proba(X_test_bin_scaled)
# 
# print(f"F1-score: {f1_score(y_test_bin, y_pred):.4f}")
# print(f"Precision: {precision_score(y_test_bin, y_pred):.4f}")
# print(f"Recall: {recall_score(y_test_bin, y_pred):.4f}")